# mainSimulation

In [1]:
import os
import pickle
import numpy as np
import itertools
import pandas as pd
from setMain import model2attBSIM, main4_updated, dict2df
import time

checked = []; noresult = []; noresultP = [];
networks = ['n10toy_Network']

for networkName in networks:
    model_file = '/example/' +networkName+'.bnet'  
    simulationPickle = '/example/' +networkName+'.pickle' 

    start = time.time()
    str_steadyO, D_listO, U_listO, desiredO, undesiredO, phenotypeNodeO = model2attBSIM(model_file, simulationPickle, n_clusters=2, pThres=1)
    
    # User defined (example) =============================
    # desiredO = ['**111*00*1', '0011100001']
    # undesiredO = set(str_steadyO) - set(desiredO)
    # phenotypeNodeO = ['~x07','x03']
    # ====================================================
    if len(phenotypeNodeO)<2 : 
        noresultP.append(networkName)
        continue


    # main algorithm
    V_idx, allresult, allremained = main4_updated(model_file, desiredO, undesiredO, phenotypeNodeO, oriN=0, remainedThres=10)

    renamed = {v:k for k,v in V_idx.items()}
    mergedRDF = dict2df(renamed, allresult, [])
    
    # iteratively apply main algorithm 
    for candk, netI in allremained.items():
        plusCtrl = candk.split(':')
        while 1:
            model_file, _, desired, undesired, phenotypeNode = netI[0]
            if len(phenotypeNode)<2 : break
            _, allresult2, allremained2 = main4_updated(model_file, desired, undesired, phenotypeNode, oriN=int(len(V_idx)/2), remainedThres=10)
            rdf2 = dict2df(renamed, allresult2, plusCtrl)
            mergedRDF = pd.concat([mergedRDF,rdf2])
            if len(allremained2) == 0 :break
            else: # next remained network
                nextk = list(allremained2.items())[0][0].split(':')
                model_file, _, desired, undesired, phenotypeNode = list(allremained2.items())[0][1][0]
                plusCtrl += nextk
            


    if mergedRDF.shape[0] == 0:
        print('NO RESULT')
        noresult.append(networkName)
    else:
        mergedRDF = mergedRDF.loc[[len(x)!=0 for x in mergedRDF.solution],:]
        # save result ===================================================
        # savedir = '/script/set_result/'
        # with open(savedir+networkName+'_result.pickle','wb') as f:
        #     pickle.dump((networkName, str_steadyO, D_listO, U_listO, desiredO, undesiredO, phenotypeNodeO, mergedRDF), f)
        # ===============================================================
        min_sols = np.min([np.min([len(y) for y in x]) for x in mergedRDF.solution])
        mergedRDF2 = mergedRDF.iloc[np.where([np.min([len(y) for y in x]) == min_sols for x in mergedRDF.solution])[0],:]
        numD_ = max(mergedRDF2.num_D) # the number of desired states by control target 
        ctrl_ = max(mergedRDF2.ctrl) # predetermined control target
        sol_ = set(itertools.chain(*mergedRDF2.solution))

        checked.append((networkName, numD_, sol_, np.min([len(x) for x in sol_]), time.time()-start))

Matched? True
/script/boolsim/network/n10toy2_Network.bnet
Cluster# 2 [1 0 0]


In [2]:
# Output ==========================================================
# numD_ : the number of induced desired states after applying the control inputs 
# sol_ : control inputs
# Size : the number of elements in control inputs
# Time : computational time
# =================================================================

checked2 = pd.DataFrame(checked)
checked2.index = checked2.iloc[:,0]
checked2.columns = ['networkName', 'numD_', 'sol_', 'Size','Time']
checked2.loc[:,['numD_', 'sol_', 'Size','Time']]

,numD_,sol_,Size,Time
0,,,,
n10toy2_Network,2,"{(~x01, ~x04), (~x01, x02)}",2,0.095129
